**<span style="color:#A03;font-size:20pt">
&#x1f3af; Assignment 7
</span>**

**Before you get started, please read the instructions below:**

>**1)** Don't feel intimidated by the length of this notebook which is caused by the detailed instructions given to make things clear and straightforward. The amount of coding that you need to do is very small.
>
>**2)** Go through this notebook and read the instructions and comments carefully to complete the `# TODO:` items (the only coding you need to do is addressing `# TODO:` items) and answer the questions. 
>
>**3)** Please do **NOT** delete any part(s) of markdown or code blocks, including the `# TODO:` instructions. Just add your answers, code, and comments to the notebook.
>
>**4) Make sure conda environment `chem413` is activated, then install `sympy` library. Launch this Jupyter notebook is launched from `chem413` environment. The libraries that might need (`numpy`, `matplotlib`, `scipy`, and `sympy`) should be already installed in this environment.**

In [1]:
# Execute this code block to import libraries

import sympy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Execute this code block

class HuckelModel(object):
    """Huckel Model."""

    def __init__(self, adjacency, alpha=None, beta=None):
        """Initialize class."""
        if not adjacency.ndim == 2:
            raise ValueError(f"Adjacency matrix should be a 2-dimensional array; got adjacency.ndim={adjacency.ndim}")

        self._number_nodes = len(adjacency)
        self._alpha = alpha
        self._beta = beta

        # make hamiltonian matrix and find eigenvalues and eigenvectors
        if (alpha is None) or (beta is None):
            alpha, beta = sympy.symbols('a, b')
            hamiltonian = beta * adjacency
            np.fill_diagonal(hamiltonian, alpha)
            self._hamiltonian = sympy.Matrix(hamiltonian)
            self._eigenvalues = self._hamiltonian.eigenvals()
            self._eigenvectors = self._hamiltonian.eigenvects()

        else:
            self._hamiltonian = beta * adjacency
            np.fill_diagonal(self._hamiltonian, alpha)
            # eigenvectors are already normalized
            self._eigenvalues, self._eigenvectors = np.linalg.eigh(self._hamiltonian)
            assert np.all(np.isreal(self._eigenvalues))
            assert np.all(np.isreal(self._eigenvectors))
            assert np.all(self._eigenvalues == sorted(self._eigenvalues))
            # check that orbitals are orthonormal
            assert np.all(abs(np.dot(self._eigenvectors.T, self._eigenvectors) - np.eye(self._number_nodes)) < 1.e-6)
            #assert np.all(abs(np.linalg.norm(self._eigenvectors, axis=0) - 1.0) < 1.e-6)

    @property
    def hamiltonian(self):
        """Hamiltonian matrix."""
        return np.array(self._hamiltonian)

    @property
    def overlap(self):
        """Overlap matrix."""
        return np.array(sympy.eye(self._number_nodes))

    @property
    def energies(self):
        """Molecular orbital energies."""
        return self._eigenvalues

    @property
    def coefficients(self):
        """Molecular orbital coefficients."""
        return self._eigenvectors

    def compute_energy(self, n_alpha, n_beta):
        """Compute total energy of the molecule given the number of alpha and beta electrons."""
        if self._alpha is None or self._beta is None:
            print("Cannot compute energy when the values of alpha and beta are not give!")
            return
        energy = np.sum(self._eigenvalues[:n_alpha])
        energy += np.sum(self._eigenvalues[:n_beta])
        return energy

    def compute_populations(self, n_alpha, n_beta):
        """Compute atomic populations."""
        raise NotImplementedError()


**<span style="color:#A03;font-size:20pt">
H&uuml;ckel Method
</span>**

In this assignment you will apply the linear combination of basis functions and secular equations to determine the molecular orbital energies of $\pi$-electrons in conjugated hydrocarbons. In H&uuml;ckel model: 

- The basis functions are pi-orbitals (perpendicular to the molecular plane) placed at the position of atomic nuclei (i.e., one pi-orbital per carbon atom). 

- The elements of overlap matrix are defined to be: $S_{ii}=1$ and $S_{ij}=0$ for $i \neq j$. In other words, the overlap matrix is diagonal; atomic orbitals are normalized and there is no overlap between two atomic orbitals (on two different centers).

- The elements of Hamiltonian matrix are defined to be: $H_{ii}=\alpha$ and $H_{ij}=\beta$ for adjacent atoms $i$ and $j$ where $i \neq j$. That is, unless atoms $i$ and $j$ are connected by a $\sigma$-bond, the Hamiltonian matrix elements are zero.

[Click here](https://en.wikipedia.org/wiki/H%C3%BCckel_method) to read more about the H&uuml;ckel method.

The above code-block has fully implemented the `HuckelModel` class which you will use to study various conjugated hydrocarbons. To use this code, you need to define an [adjacency matrix](https://en.wikipedia.org/wiki/Adjacency_matrix) for your molecule, which captures the connectivity between Carbon atoms. That is, for a molecule with $N$ Carbon atoms, the adjacency matrix is an $N\times N$ symmetric matrix with 1 and 0 elements specifying whether two Carbon atoms are connected with a $\sigma$ bond or not, respectively.

**<span style="color:#A03;font-size:16pt">
Example: Ethylene
</span>**

For ethylene, the adjacency matrix is a $2 \times 2$ symmetric matrix with 0 for diagonal and 1 for off-diagonal elements; see in the code-block below. Based on your learned in the last lecture, write down (on a piece of paper or digital document) the equations you are solving for ethylene based on H&uuml;ckel model. Then study the the code block below and the values printed to see whether they are as expected. Specifically,

- Do the Hamiltonian and Overlap matrices look as you expected?
- Do the orbital energies look as you expected?
- Does the total energy match what you obtain from the your pen-and-paper calculation?
- Can you interpret the orbital coefficients? How do the orbitals look like?
- How you would compute the ionization potential of ethylene based on the computed molecular orbitals? Does it match the experimental value?
- How you would compute the excitation energy of ethylene based on the computed molecular orbitals? Does it match the experimental value?

**NOTE:** When values of `alpha` and `beta` arguments are `None`, the code treats them as symbols but still solves the equations. In this case, the `energies` attribute is a dictionary with keys corresponding to molecular orbitals energies (i.e., `a - b` and `a + b`) and values corresponding to their degeneracy (i.e., `1` for both orbitals). However, if values are `alpha` and `beta` are given when making an instance of `HuckelMolde`, the `energies` attribute returns the numerical values of molecular orbital energies.

**NOTE:** The `coefficients` attribute is a $2 \times 2$ matrix where its columns contain the coefficients of molecular orbitals. That is, the 1st column contains the coefficients of the first molecular orbital.

In [3]:
# adjacency matrix of ethylene
adj = np.array([[0, 1],
                [1, 0]])

# make an instance of HuckelModel (without assigning a value to alpha and beta)
h = HuckelModel(adj, alpha=None, beta=None)

print("Hamiltonian Matrix:")
print(h.hamiltonian)
print("")
print("Overlap Matrix:")
print(h.overlap)
print("")
print("Energies:")
print(h.energies)
print("")

print("---" * 20)

# make an instance of HuckelModel (alpha and beta are provided in Hartree)
h = HuckelModel(adj, alpha=-0.414, beta=-0.0533)

print("Hamiltonian Matrix:")
print(h.hamiltonian)
print("")
print("Overlap Matrix:")
print(h.overlap)
print("")
print("Orbital Energies:")
print(h.energies)
print("")
print("Total Energy (for 1 alpha and 1 beta electron):")
print(h.compute_energy(1, 1))
print("")
print("Orbital Coefficients:")
print(h.coefficients)

Hamiltonian Matrix:
[[a b]
 [b a]]

Overlap Matrix:
[[1 0]
 [0 1]]

Energies:
{a - b: 1, a + b: 1}

------------------------------------------------------------
Hamiltonian Matrix:
[[-0.414  -0.0533]
 [-0.0533 -0.414 ]]

Overlap Matrix:
[[1 0]
 [0 1]]

Orbital Energies:
[-0.4673 -0.3607]

Total Energy (for 1 alpha and 1 beta electron):
-0.9346

Orbital Coefficients:
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


**<span style="color:#A03;font-size:20pt">
&#129300; Question 1: 1,3-butadiene
</span>** 

In the code block below, write the adjacency matrix for 1,3-butadiene.

**<span style="color:#A03;font-size:14pt">
Question 1.1
</span>**

> In the code block below, use the H&uuml;ckel model to compute its orbital energies (without assigning the `alpha` and `beta` parameters). Draw the molecular orbital diagram of 1,3-butadiene. What is the total energy of 1,3-butadiene as a function of `alpha` and `beta`?

PUT YOUR ANSWER HERE.


**<span style="color:#A03;font-size:14pt">
Question 1.2
</span>**

> In the code block below, use the H&uuml;ckel model to compute its orbital energies assuming that `alpha=-0.414 a.u.`, `beta=-0.0533 a.u.`. What is the total energy of 1,3-butadiene?

PUT YOUR ANSWER HERE.


**<span style="color:#A03;font-size:14pt">
Question 1.3
</span>**

> How do the occupied orbitals of 1,3-butadiene look like based on the orbital coefficients?

PUT YOUR ANSWER HERE.

In [ ]:
# 1,3-butadiene

**<span style="color:#A03;font-size:20pt">
&#129300; Question 2: Cyclobutadiene
</span>** 

In the code block below, write the adjacency matrix for cyclobutadiene.

**<span style="color:#A03;font-size:14pt">
Question 2.1
</span>**

> In the code block below, use the H&uuml;ckel model to compute its orbital energies (without assigning the `alpha` and `beta` parameters). Draw the molecular orbital diagram of cyclobutadiene. How does the orbital energies/diagram differ from 1,3-butadiene?

PUT YOUR ANSWER HERE.


**<span style="color:#A03;font-size:14pt">
Question 2.2
</span>**

> In the code block below, use the H&uuml;ckel model to compute its orbital energies assuming that `alpha=-0.414 a.u.`, `beta=-0.0533 a.u.`. What is the total energy of cyclobutadiene? How does this compare to the energy of 1,3-butadiene?

PUT YOUR ANSWER HERE.


**<span style="color:#A03;font-size:14pt">
Question 2.3
</span>**

> How do the occupied orbitals of cyclobutadiene look like based on the orbital coefficients?

PUT YOUR ANSWER HERE.

In [ ]:
# cyclobutadiene 

**<span style="color:#A03;font-size:20pt">
&#x1f3af; End of Assignment 7
</span>**